In [2]:
import pandas as pd
import numpy as np

np.random.seed(42)

n = 5000  # número de hospitalizaciones

df = pd.DataFrame({
    "encounter_id": range(1, n + 1),
    "age": np.random.randint(18, 90, n),
    "sex": np.random.choice(["Male", "Female"], n),
    "payer": np.random.choice(
        ["Medicare", "Medicaid", "Private", "Self-pay"],
        n,
        p=[0.4, 0.2, 0.3, 0.1]
    ),
    "admission_type": np.random.choice(
        ["Emergency", "Elective", "Urgent"],
        n,
        p=[0.6, 0.25, 0.15]
    ),
    "length_of_stay": np.random.poisson(lam=4, size=n) + 1,
    "diagnosis_group": np.random.choice(
        ["Cardiology", "Endocrinology", "Pulmonology", "Orthopedics", "Neurology"],
        n
    )
})

base_cost = np.random.lognormal(mean=8, sigma=0.7, size=n)

df["total_charges"] = (
    base_cost
    * (1 + df["length_of_stay"] * 0.15)
    * np.where(df["admission_type"] == "Emergency", 1.4, 1.0)
)

df.head()

,encounter_id,age,sex,payer,admission_type,length_of_stay,diagnosis_group,total_charges
0,1,69,Male,Medicare,Elective,4,Pulmonology,4384.555564
1,2,32,Female,Medicare,Elective,6,Orthopedics,4559.744021
2,3,89,Male,Medicare,Elective,3,Cardiology,3014.315546
3,4,78,Male,Private,Emergency,7,Endocrinology,20795.605045
4,5,38,Male,Private,Emergency,4,Endocrinology,6052.970564


In [1]:
import duckdb
import pandas as pd
import numpy as np

print("duckdb:", duckdb.__version__)
print("pandas:", pd.__version__)
print("numpy:", np.__version__)

duckdb: 1.4.4
pandas: 3.0.1
numpy: 2.4.2


In [4]:
output_path = "../data/processed/encounters_clean.csv"
df.to_csv(output_path, index=False)
print("Save in:", output_path)

OSError: Cannot save file into a non-existent directory: '..\data\processed'

In [5]:
import os

os.getcwd()

'c:\\Users\\Ilan\\Documents\\Hospital Cost Drivers\\Notebooks'

In [6]:
os.listdir("..")

['.gitignore',
 '.venv',
 'DATA',
 'Notebooks',
 'README.md',
 'Reports',
 'Requirements.txt',
 'SQL',
 'src']

In [7]:
os.listdir("../data")

['Raw']

In [8]:
df.to_csv("../data/processed/encounters_clean.csv", index=False)
print("CSV correctly saved")

CSV correctly saved


In [9]:
import duckdb

# create local base DuckDB 
con = duckdb.connect("health.db")
con

In [10]:
con.execute("""
CREATE OR REPLACE TABLE encounters AS
SELECT *
FROM read_csv_auto('../data/processed/encounters_clean.csv')
""")

In [11]:
con.execute("SELECT COUNT(*) FROM encounters").fetchall()

[(5000,)]

In [12]:
con.execute("SELECT * FROM encounters LIMIT 5").df()

,encounter_id,age,sex,payer,admission_type,length_of_stay,diagnosis_group,total_charges
0,1,69,Male,Medicare,Elective,4,Pulmonology,4384.555564
1,2,32,Female,Medicare,Elective,6,Orthopedics,4559.744021
2,3,89,Male,Medicare,Elective,3,Cardiology,3014.315546
3,4,78,Male,Private,Emergency,7,Endocrinology,20795.605045
4,5,38,Male,Private,Emergency,4,Endocrinology,6052.970564


In [13]:
pareto = con.execute("""
WITH cost_by_patient AS (
    SELECT
        encounter_id,
        SUM(total_charges) AS patient_cost
    FROM encounters
    GROUP BY encounter_id
),
ranked AS (
    SELECT
        encounter_id,
        patient_cost,
        SUM(patient_cost) OVER () AS total_cost,
        SUM(patient_cost) OVER (
            ORDER BY patient_cost DESC
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cumulative_cost
    FROM cost_by_patient
)
SELECT
    encounter_id,
    patient_cost,
    cumulative_cost / total_cost AS cumulative_cost_pct
FROM ranked
ORDER BY patient_cost DESC
""").df()

pareto.head(10)

,encounter_id,patient_cost,cumulative_cost_pct
0,1556,64430.760061,0.001547
1,2484,61496.190001,0.003024
2,1615,61042.086365,0.004489
3,2002,59987.335179,0.005929
4,1012,59106.463152,0.007349
5,3221,59089.197278,0.008767
6,1699,58382.638290,0.010169
7,3109,56143.788286,0.011517
8,2340,54429.497109,0.012824
9,1801,54211.338011,0.014126


In [14]:
pareto[pareto["cumulative_cost_pct"] <= 0.6].shape[0] / pareto.shape[0]

0.3124

In [15]:
pareto.to_csv("../reports/tables/pareto_high_cost_patients.csv", index=False)

In [16]:
import os

os.makedirs("../reports/tables", exist_ok=True)

In [18]:
pareto.to_csv("../reports/tables/pareto_high_cost_patients.csv", index=False)
print("Pareto coreclty saved")

Pareto coreclty saved


In [19]:
import pandas as pd

pd.read_csv("../reports/tables/pareto_high_cost_patients.csv").head(5)

,encounter_id,patient_cost,cumulative_cost_pct
0,1556,64430.760061,0.001547
1,2484,61496.190001,0.003024
2,1615,61042.086365,0.004489
3,2002,59987.335179,0.005929
4,1012,59106.463152,0.007349


In [20]:
# % de encounters que generan ~60% del costo
pct_60 = (
    pareto[pareto["cumulative_cost_pct"] <= 0.60].shape[0]
    / pareto.shape[0]
)

pct_60

0.3124

In [21]:
# mark high-cost encounters (top 30%)
high_cost_threshold = pareto["patient_cost"].quantile(0.70)

high_cost_threshold

np.float64(9389.35834512158)

In [22]:
high_cost_df = con.execute("""
SELECT
  *
FROM encounters
WHERE total_charges >= (
    SELECT
      quantile_cont(total_charges, 0.70)
    FROM encounters
)
""").df()

high_cost_df.head()

,encounter_id,age,sex,payer,admission_type,length_of_stay,diagnosis_group,total_charges
0,4,78,Male,Private,Emergency,7,Endocrinology,20795.605045
1,11,47,Female,Medicare,Urgent,6,Endocrinology,10651.035297
2,18,75,Female,Medicare,Emergency,4,Endocrinology,10852.149192
3,21,76,Male,Private,Emergency,4,Pulmonology,9813.817607
4,23,77,Male,Medicaid,Emergency,4,Pulmonology,12041.926823


In [23]:
high_cost_df.shape

(1500, 8)

In [24]:
cohorts = con.execute("""
SELECT
  diagnosis_group,
  admission_type,
  payer,
  CASE
    WHEN length_of_stay <= 2 THEN '0-2 days'
    WHEN length_of_stay <= 5 THEN '3-5 days'
    WHEN length_of_stay <= 10 THEN '6-10 days'
    ELSE '11+ days'
  END AS los_bucket,
  COUNT(*) AS encounters,
  AVG(total_charges) AS avg_charges,
  SUM(total_charges) AS total_charges
FROM encounters
WHERE total_charges >= (
    SELECT quantile_cont(total_charges, 0.70)
    FROM encounters
)
GROUP BY
  diagnosis_group,
  admission_type,
  payer,
  los_bucket
ORDER BY total_charges DESC
""").df()

cohorts.head(10)

,diagnosis_group,admission_type,payer,los_bucket,encounters,avg_charges,total_charges
0,Neurology,Emergency,Medicare,6-10 days,43,18331.924837,788272.768003
1,Cardiology,Emergency,Medicare,6-10 days,45,17304.197392,778688.882648
2,Endocrinology,Emergency,Medicare,6-10 days,39,19597.692149,764309.993794
3,Cardiology,Emergency,Medicare,3-5 days,42,16732.146583,702750.156490
4,Pulmonology,Emergency,Medicare,3-5 days,40,17539.736390,701589.455603
5,Neurology,Emergency,Medicare,3-5 days,41,16399.465039,672378.066587
6,Endocrinology,Emergency,Medicare,3-5 days,37,17340.326901,641592.095324
7,Orthopedics,Emergency,Medicare,6-10 days,38,16649.364402,632675.847289
8,Pulmonology,Emergency,Medicare,6-10 days,44,14053.411266,618350.095699
9,Cardiology,Emergency,Private,6-10 days,33,17211.048432,567964.598242


In [26]:
cohorts.to_csv("../reports/tables/high_cost_cohorts.csv", index=False)
print("Saved: high_cost_cohorts.csv")

Saved: high_cost_cohorts.csv


In [28]:
#High-cost encounters cluster around emergency admissions for Medicare patients with neurological and cardiovascular conditions, particularly when length of stay exceeds 3–5 days.

In [29]:
con.close()
print("DuckDB connection closed")

DuckDB connection closed
